#Import


In [1]:
!pip install numpy -q
!pip install pandas -q
!pip install matplotlib -q
!pip install tensorflow -q

!pip install opendatasets -q

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import os
import opendatasets as od


# Directories

In [3]:
od.download("https://www.kaggle.com/datasets/msambare/fer2013")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: isurusanjeewa
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013


# Directories

In [4]:
train_data_dir = "/content/fer2013/train"
validation_data_dir = "/content/fer2013/test"

# Data generators

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,

    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Data loaders

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.



# Class labels

In [8]:
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Fetch a batch of images and labels (optional step for debugging)

In [9]:
img, label = train_generator.__next__()

# Model architecture

In [21]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compile model

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Print model summary

In [12]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 913,159 (3.48 MB)

 Trainable params: 913,159 (3.48 MB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:
train_path = "/content/fer2013/train"
test_path ="/content/fer2013/test"

In [14]:
num_train_imgs =0
for root, dirs,files in os.walk(train_path) :
    num_train_imgs += len(files)

num_test_imgs =0
for root, dirs, files in os.walk(test_path):
    num_test_imgs += len(files)


epochs=100

history=model.fit(train_generator,
                  steps_per_epoch=num_train_imgs//32,
                  epochs=epochs,
                  validation_data =validation_generator,
                  validation_steps=num_test_imgs//32)




Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


897/897 ━━━━━━━━━━━━━━━━━━━━ 27s 21ms/step - accuracy: 0.2700 - loss: 1.7722 - val_accuracy: 0.4358 - val_loss: 1.4657
Epoch 2/100
  1/897 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.4375 - loss: 1.3423

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


897/897 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4375 - loss: 1.3423 - val_accuracy: 0.4339 - val_loss: 1.4693
Epoch 3/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.4425 - loss: 1.4395 - val_accuracy: 0.5047 - val_loss: 1.2853
Epoch 4/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.3750 - loss: 1.5045 - val_accuracy: 0.5068 - val_loss: 1.2816
Epoch 5/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.5020 - loss: 1.2954 - val_accuracy: 0.5335 - val_loss: 1.2121
Epoch 6/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4375 - loss: 1.3353 - val_accuracy: 0.5332 - val_loss: 1.2120
Epoch 7/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - accuracy: 0.5418 - loss: 1.2094 - val_accuracy: 0.5386 - val_loss: 1.1984
Epoch 8/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5312 - loss: 1.1117 - val_accuracy: 0.5370 - val_loss: 1.2018
Epoch 9/100
897/897 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.5470 - loss: 1.1769 - val_accuracy

In [15]:
model.save('ai_project_model1_file_v6.h5')

Test accuracy


In [16]:
loss, accuracy = model.evaluate(validation_generator, steps=num_test_imgs // 32)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

224/224 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6154 - loss: 1.2281
Test Loss: 1.2443
Test Accuracy: 0.6147


Train accuracy

In [17]:
# Get final training accuracy
final_train_accuracy = history.history['accuracy'][-1]
print(f"Final Training Accuracy: {final_train_accuracy:.4f}")


Final Training Accuracy: 0.7188
